In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
import os
import json
from PIL import Image
from tqdm import tqdm

# Пути к папкам с изображениями и аннотациями (txt-файлы)
images_dir = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images"
labels_dir = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/labels"
output_json = '/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json'

# Список изображений (расширения можно дополнять)
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f'count of images: {len(image_files)}')

# Определяем категории (пример: 0 - корабль, 1 - самолёт, 2 - машина)
categories = [
    {"id": 1, "name": "plane", "supercategory": "none"},
    {"id": 2, "name": "car", "supercategory": "none"},
    {"id": 3, "name": "ship", "supercategory": "none"}
]

images = []
annotations = []
ann_id = 1  # уникальный идентификатор для каждой аннотации
path_not_found_counter = 0

for img_id, file_name in tqdm(enumerate(image_files)):
    # if img_id > 20:
    #   break

    img_path = os.path.join(images_dir, file_name)
    # Открываем изображение для получения размеров
    with Image.open(img_path) as img:
        width, height = img.size

    # Формируем информацию об изображении
    image_info = {
        "id": img_id,
        "width": width,
        "height": height,
        "file_name": file_name
    }
    images.append(image_info)
    # print(f'len images: {len(images)}')

    # Путь к соответствующему txt-файлу с аннотациями (предполагается, что имя совпадает)
    label_file = os.path.join(labels_dir, os.path.splitext(file_name)[0] + '.txt')
    # print(f'filename {file_name}')
    # print(f'label file {label_file}')
    # print(f'exist {os.path.exists(label_file)}')
    if os.path.exists(label_file):
        line_count = 0
        with open(label_file, 'r') as f:
            lines = f.readlines()
        for line in lines:
            line = line.strip()
            if not line:
                continue
            line_count +=1
            # Разбираем строку формата: <class> <x_center> <y_center> <width> <height>
            parts = line.split()
            if len(parts) != 5:
                continue
            class_id, x_center, y_center, w_norm, h_norm = map(float, parts)
            # Преобразуем нормализованные значения в абсолютные координаты
            x_center_abs = x_center * width
            y_center_abs = y_center * height
            w_abs = w_norm * width
            h_abs = h_norm * height
            # Переводим в формат COCO: [x_min, y_min, width, height]
            x_min = x_center_abs - w_abs / 2
            y_min = y_center_abs - h_abs / 2

            annotation = {
                "id": ann_id,
                "image_id": img_id,
                "category_id": int(class_id),
                "bbox": [x_min, y_min, w_abs, h_abs],
                "area": w_abs * h_abs,
                "iscrowd": 0
            }
            annotations.append(annotation)
            ann_id += 1

        if line_count == 0:
          print(f"Пустой файл: {label_file}")

    else:
      path_not_found_counter += 1
      if path_not_found_counter % 100 == 0:
        print(f'Путь не найден: {path_not_found_counter}')


print(f'ann id: {ann_id}')
# Собираем итоговую структуру в формате COCO
coco_format = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

# Сохраняем в JSON-файл
with open(output_json, 'w') as f:
    json.dump(coco_format, f, indent=4)

print(f'Не найденных лейблов: {path_not_found_counter}')
print(f"Обработано изображений: {len(images)}")
print(f"Найдено аннотаций: {len(annotations)}")
print(f"Количество аннотационных файлов: {len(os.listdir(labels_dir))}")

print(f"COCO-аннотации сохранены в {output_json}")


count of images: 3714


46it [01:10,  2.14s/it]

Пустой файл: /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/labels/b9438e44-6980-492a-becb-bc9017f323ff.txt


92it [01:10,  2.78it/s]

Пустой файл: /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/labels/b63e09d4-4c22-42a3-8b20-bde3c6ed3025.txt


...

Пустой файл: /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/labels/1f082c6a-77ec-40c6-8837-90c191c74a30.txt
Пустой файл: /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/labels/18fead09-4d27-4e07-ab0c-369fbc540e24.txt


3622it [25:35,  2.80it/s]

Пустой файл: /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/labels/2ac0bda0-7b3f-46c3-803e-a955a913a8a7.txt


3714it [26:11,  2.36it/s]


ann id: 17340
Не найденных лейблов: 6
Обработано изображений: 3714
Найдено аннотаций: 17339
Количество аннотационных файлов: 3708
COCO-аннотации сохранены в /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json


In [4]:
import json
import os
import random

# Путь к файлу с аннотациями COCO
annotations_path = output_json
train_output_path = output_json
val_output_path = '/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json'

# Доля данных для валидации
val_ratio = 0.1

# Чтение исходного файла с аннотациями
with open(annotations_path, 'r') as f:
    data = json.load(f)

# Перемешиваем изображения для случайного выбора валидационных
random.shuffle(data['images'])

# Вычисляем количество валидационных изображений
val_size = int(len(data['images']) * val_ratio)

# Разделяем изображения на тренировочные и валидационные
val_images = data['images'][:val_size]
train_images = data['images'][val_size:]

# Получаем ID валидационных изображений
val_image_ids = {img['id'] for img in val_images}

# Разделяем аннотации на тренировочные и валидационные
val_annotations = [ann for ann in data['annotations'] if ann['image_id'] in val_image_ids]
train_annotations = [ann for ann in data['annotations'] if ann['image_id'] not in val_image_ids]

# Формируем тренировочный и валидационный словари в формате COCO
train_data = {
    "images": train_images,
    "annotations": train_annotations,
    "categories": data['categories']
}

val_data = {
    "images": val_images,
    "annotations": val_annotations,
    "categories": data['categories']
}

with open(train_output_path, 'w') as f:
    json.dump(train_data, f, indent=4)

with open(val_output_path, 'w') as f:
    json.dump(val_data, f, indent=4)

print(f"Тренировочные данные сохранены в {train_output_path}")
print(f"Валидационные данные сохранены в {val_output_path}")


Тренировочные данные сохранены в /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json
Валидационные данные сохранены в /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json
